## imputing surrogate index from historical data

In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(library(caret)))

In [2]:
dir_path = '/export/projects2/jeryang_narrative_project/globe/log'
setwd(dir_path)

### use historical data from 2015 - 2018 to estimate surrogate index 

In [3]:
history2015 = read.csv('./history2015.csv')

In [4]:
xgb_surrogate <- caret::train(rev ~., # estimate surrogate index on 3-year revenue
                              #rev_18m ~., # estimating surrogate index on 18-month revenue (for validation)
                   data = select(history2015, -churn, -last_stop_date, -subscriber_id),
                              #data = select(history2015, rev_18m, matches('_1|_m1|_2|_m2|_3|_m3|_4|_m4|_5|_m5|_6|_m6')), #using surrogates from the first 6 months
                              #data = select(history2015, rev_18m, matches('_1|_m1|_2|_m2|_3|_m3|_4|_m4|_5|_m5')), #using surrogates from the first 5 months
                              #data = select(history2015, rev_18m, matches('_1|_m1|_2|_m2|_3|_m3|_4|_m4')), #using surrogates from the first 4 months
                              #data = select(history2015, rev_18m, matches('_1|_m1|_2|_m2|_3|_m3')), #using surrogates from the first 3 months
                              #data = select(history2015, rev_18m, matches('_1|_m1|_2|_m2')), #using surrogates from the first 2 months
                              #data = select(history2015, rev_18m, matches('_1|_m1')), #using surrogates from the first 1 month
                   method = "xgbTree",
                             tuneGrid = data.frame(eta = 0.3,
                                      max_depth = 2,
                                      gamma = 0,
                                      colsample_bytree = 0.8,
                                      min_child_weight = 1,
                                      subsample = 1,
                                      nrounds = 150))

### impute surrogate index in the experiment

In [5]:
df1_s = read.csv('./df_surrogate_exp1.csv') # first experiment 
#df2_s = read.csv('df_surrogate_exp2.csv') # second experiment 

In [6]:
df1_s$rev3 = predict(xgb_surrogate,
                              newdata =  select(df1_s, -subscriber_id, -treated, -p_treated, -rev, -rev3))

In [7]:
head(df1_s$rev3)

[1]  835.5392 1022.2662  844.0248  787.8553  788.8301  809.9722